In [1]:
%%time

!apt-get update 1>/dev/null 2>/dev/null
!apt-get install chromium-chromedriver 1>/dev/null 2>/dev/null
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium 1>/dev/null 2>/dev/null

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import math
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
BASE_URL = 'https://db.netkeiba.com/'
endpoint = '?pid=race_search_detail'

In [4]:
!pip install tqdm
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
from tqdm import tqdm_notebook as tqdm
import re
import pandas as pd

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# /usr/lib/chromium-browser/chromedriver
driver = webdriver.Chrome('chromedriver', options=options)
driver.get(BASE_URL + endpoint)

In [6]:
import copy

driver2 = copy.copy(driver)

In [7]:
# driver.get('https://race.netkeiba.com/race/result.html?race_id=202005030401&rf=race_list')
# driver2.get('https://race.netkeiba.com/odds/index.html?race_id=202005030401&rf=race_submenu')

In [8]:
%%time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# /usr/lib/chromium-browser/chromedriver
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(3)
driver.get(BASE_URL + endpoint)

### 以下の項目で収穫対象をunコメントしてください。

## 競走種別
driver.find_element_by_id('check_track_1').click() # 芝
driver.find_element_by_id('check_track_2').click() # ダート
driver.find_element_by_id('check_track_3').click() # 障害

## 期間
links_num = driver.find_element_by_name('list')
nums_element = Select(links_num)
# nums_element.select_by_value("20")
# nums_element.select_by_value("50")
nums_element.select_by_value("100")

## 競馬場
# driver.find_element_by_id('check_Jyo_01').click() # 札幌
# driver.find_element_by_id('check_Jyo_02').click() # 函館
# driver.find_element_by_id('check_Jyo_03').click() # 福島
# driver.find_element_by_id('check_Jyo_04').click() # 新潟
driver.find_element_by_id('check_Jyo_05').click() # 東京
# driver.find_element_by_id('check_Jyo_06').click() # 中山
# driver.find_element_by_id('check_Jyo_07').click() # 中京
# driver.find_element_by_id('check_Jyo_08').click() # 京都
# driver.find_element_by_id('check_Jyo_09').click() # 阪神
# driver.find_element_by_id('check_Jyo_10').click() # 小倉

## 馬場状態
# driver.find_element_by_id('check_baba_1').click() # 良
# driver.find_element_by_id('check_baba_2').click() # 稍重
# driver.find_element_by_id('check_baba_3').click() # 重
# driver.find_element_by_id('check_baba_4').click() # 不良

## 条件
# driver.find_element_by_id('check_jyoken_1').click() # 牝馬限定
# driver.find_element_by_id('check_jyoken_2').click() # 父内国産限定
# driver.find_element_by_id('check_jyoken_3').click() # アラブ
# driver.find_element_by_id('check_jyoken_4').click() # 指定
# driver.find_element_by_id('check_jyoken_5').click() # 混合
# driver.find_element_by_id('check_jyoken_6').click() # 特指
# driver.find_element_by_id('check_jyoken_7').click() # 国際

## 馬齢
# driver.find_element_by_id('check_barei_11').click() # ２歳
# driver.find_element_by_id('check_barei_12').click() # ３歳
# driver.find_element_by_id('check_barei_13').click() # ３歳以上
# driver.find_element_by_id('check_barei_14').click() # ４歳以上

## クラス
# driver.find_element_by_id('check_grade_1').click() # G1
# driver.find_element_by_id('check_grade_2').click() # G2
# driver.find_element_by_id('check_grade_3').click() # G3
# driver.find_element_by_id('check_grade_4').click() # OP
# driver.find_element_by_id('check_grade_5').click() # 1600万
# driver.find_element_by_id('check_grade_6').click() # 1000万
# driver.find_element_by_id('check_grade_7').click() # 500万
# driver.find_element_by_id('check_grade_8').click() # 新馬
# driver.find_element_by_id('check_grade_9').click() # 未勝利
# driver.find_element_by_id('check_grade_10').click() # 未出走

## 表示件数
links_num = driver.find_element_by_name('list')
nums_element = Select(links_num)
nums_element.select_by_value("100")
driver.find_element_by_class_name('button').send_keys(Keys.ENTER)


soup = BeautifulSoup(driver.page_source, 'html.parser')
num_pattern = '\d+'
total_race_link = re.search(num_pattern, soup.find(id="contents_liquid").find(class_='pager').text.strip().replace(',', '')).group()
total_search_page = math.ceil(int(total_race_link)/100)

all_race_links = []
filename = 'out.txt'

with open(filename, mode='w') as f:
  # for i in tqdm(range(1, total_search_page)):
  for i in tqdm(range(1, 3)):
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      pattern = '/race/\d+/'
      for a in soup.find_all('a', href=True):
        if re.search(pattern, a['href']):
          f.write(a['href'].split('/')[2] + '\n')
          all_race_links.append(a['href'].split('/')[2])
      driver.execute_script("paging('{}')".format(i+1))

df = pd.DataFrame(all_race_links, columns=['endpoint'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,endpoint
0,202005030801
1,202005030802
2,202005030803
3,202005030804
4,202005030805
...,...
195,202005020404
196,202005020405
197,202005020406
198,202005020407


In [9]:
ls

out.txt  sample_data/


In [71]:
%%time

import requests
import math
from bs4 import BeautifulSoup

class race:

    BASE_URL = 'https://race.netkeiba.com/'

    def __init__(self, race_endpoints=[]):
        race_endpoints = race_endpoints
        self.result_columns = ['race_id', '着順', '枠', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '人気', '単勝オッズ', '後3F', 'コーナー通過順', '厩舎', '馬体重(増減)']
        self.payback_columns = ['race_id', '単勝', '複勝1', '複勝2', '複勝3', '枠連', '馬連', 'ワイド1', 'ワイド2', 'ワイド3', '馬単', '3連複', '3連単']
        self.elements = []
        self.raptimes_list = [
                  'race_id', 
                  '100m', '200m', '300m', '400m', '500m', '600m', '700m', '800m', '900m',
                  '1000m', '1100m', '1200m', '1300m', '1400m', '1500m', '1600m', '1700m',
                  '1800m', '1900m', '2000m', '2100m', '2200m', '2300m', '2400m', '2500m', '2600m'
                  ]

    def fetch(self, race_id):
        html = requests.get('https://race.netkeiba.com/race/result.html?race_id=' + race_id + '&rf=race_submenu')
        return BeautifulSoup(html.content, 'lxml')

    def race_result(self, race_id):
        soup = self.fetch(race_id)
        result_table_soup = soup.find("table", {"class": 'RaceTable01 RaceCommon_Table ResultRefund Table_Show_All'})
        col_num = 15
        rows = []
        row = [race_id]
        for col in result_table_soup.find("tbody").findAll(["tr", "td"]):
            if (list(set(['FirstDisplay', 'HorseList']) & set(col['class']))):
                continue
            row.append(col.text.strip())
            # TODO
            # ここで騎手と馬のクロールを流す
            # if (col['class'] == ['Jockey']):
            # print(col.find('a')['href'])
            # if (col['class'] == ['Horse_Info']) and (col.find('a') != None):
            # print(col.find('a')['href'])
            if (col['class'] == ['Weight']):
                rows.append(row)
                row = [race_id]
        return rows

    def payback(self, race_id):
        soup = self.fetch(race_id)
        pay_table_souplen = soup.findAll("table", {"class": 'Payout_Detail_Table'})
        paybacks = [race_id]
        for sub_pay_table in pay_table_souplen:
            for col in sub_pay_table.find("tbody").findAll(["tr"]):
              if (col['class'] == ['Tansho']):
                self.elements.append('-'.join([el.text for el in col.findAll('div') if el.text != '']))
                paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
              if (col['class'] == ['Fukusho']):
                self.elements.extend([el.text for el in col.findAll('div') if el.text != ''])
                paybacks.extend(col.find(class_='Payout').text.replace(',', '').strip().split('円')[:3])
              if (col['class'] == ['Wakuren']):
                self.elements.append('-'.join([el.text for el in col.findAll('span')][:2]))
                paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
              if (col['class'] == ['Umaren']):
                self.elements.append('-'.join([el.text for el in col.findAll('span')][:2]))
                paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
              if (col['class'] == ['Wide']):
                self.elements.extend(['-'.join([el.text for el in col.findAll('span')][:6][i::3]) for i in range(3)])
                paybacks.extend(col.find(class_='Payout').text.replace(',', '').strip().split('円')[:3])
              if (col['class'] == ['Umatan']):
                self.elements.append('-'.join([el.text for el in col.findAll('span')][:2]))
                paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
              if (col['class'] == ['Fuku3']):
                self.elements.append('-'.join([el.text for el in col.findAll('span')][:3]))
                paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
              if (col['class'] == ['Tan3']):
                self.elements.append('-'.join([el.text for el in col.findAll('span')][:3]))
                paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
        return paybacks

    def raptime(self, race_id):
        soup = self.fetch(race_id)
        raptime_table_souplen = soup.find("table", {"class": 'RaceCommon_Table Race_HaronTime'})
        kyoriS = list(map(lambda y: y.text, raptime_table_souplen.find(class_='Header').findAll('th')))
        timeS = list(map(lambda y: y.text, raptime_table_souplen.find(class_='HaronTime').findAll('td')))
        raptimes = [race_id]
        raptimes.extend([None] * (len(self.raptimes_list)-1))
        for kyori, time in zip(kyoriS, timeS):
            raptimes[self.raptimes_list.index(kyori)] = time
        return raptimes

CPU times: user 53 µs, sys: 0 ns, total: 53 µs
Wall time: 56 µs


In [73]:
racer = race()
race_id = '202009040202'
data = [racer.payback(race_id), racer.elements]

pd.DataFrame(data, columns=racer.payback_columns, index=['money', 'number'])

,race_id,単勝,複勝1,複勝2,複勝3,枠連,馬連,ワイド1,ワイド2,ワイド3,馬単,3連複,3連単
money,202009040202,420,160,720,200,1110,8340,2370,480,2510,10720,11500,76230
number,1,1,17,16,1-8,1-17,1-16,17-16,1-17,1-17,1-16-17,1-17-16,None


In [74]:
rows = []
for id in all_race_links:
  rows.append(racer.payback(id))

pd.DataFrame(rows, columns=racer.payback_columns)

,race_id,単勝,複勝1,複勝2,複勝3,枠連,馬連,ワイド1,ワイド2,ワイド3,馬単,3連複,3連単
0,202005030801,530,190,240,1950,680,1950,800,7010,6340,3770,59550,231820
1,202005030802,290,160,750,820,1390,4280,1590,1590,9030,6480,41790,113020
2,202005030803,580,220,240,150,2510,2950,1300,460,850,5460,3840,19940
3,202005030804,10540,2150,260,350,3970,65970,16150,10300,1900,110060,156010,1367540
4,202005030805,480,190,340,310,1660,2820,1120,1100,2090,4200,10210,43800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,202005020404,190,110,130,170,300,390,180,350,450,590,880,2360
196,202005020405,660,200,210,190,1950,2130,600,600,670,4220,3740,21130
197,202005020406,2020,320,170,130,5520,5110,1430,690,300,14510,2910,41990
198,202005020407,630,160,120,140,860,1020,340,430,320,2400,1240,8850


In [75]:
racer = race()
race_id = '202009040202'
pd.DataFrame(racer.race_result(race_id), columns=racer.result_columns)

,race_id,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,馬体重(増減)
0,202009040202,1,1,1,カケル,牡3,56.0,幸,1:35.9,,2,4.2,36.3,4-5,栗東大根田,482(-4)
1,202009040202,2,8,17,アドマイヤパンドラ,牝3,54.0,浜中,1:36.0,1/2,10,43.6,35.4,16-15,栗東須貝,452(+8)
2,202009040202,3,8,16,ワールドスケール,牡3,56.0,小崎,1:36.2,1.1/4,3,5.5,36.8,2-2,栗東野中,466(-8)
3,202009040202,4,8,15,ガレットデロワ,牝3,53.0,☆富田,1:36.2,クビ,11,43.9,36.1,12-9,栗東池添兼,402(-4)
4,202009040202,5,5,10,レッドエーデル,牝3,54.0,酒井,1:37.1,5,9,39.9,36.8,14-11,栗東斉藤崇,448(0)
5,202009040202,6,4,8,レヴィオーサ,牡3,56.0,和田,1:37.3,1.1/4,4,13.4,37.9,9-2,栗東友道,450(0)
6,202009040202,7,7,14,サンテルノ,牡3,56.0,北村友,1:37.3,クビ,8,38.5,37.5,9-7,栗東池添学,514(+12)
7,202009040202,8,4,7,パズル,牡3,56.0,藤井勘,1:37.5,1,6,20.1,37.8,5-5,栗東池江,490(-4)
8,202009040202,9,7,13,ハーランズハーツ,牡3,56.0,松若風,1:37.8,1.3/4,7,25.5,38.0,5-7,栗東橋口,432(-2)
9,202009040202,10,6,11,モルトルバート,セ3,56.0,川田,1:37.8,ハナ,5,15.6,37.0,14-16,栗東松永幹,432(-4)


In [76]:
racer = race()
pd.DataFrame([racer.payback(race_id)], columns=racer.payback_columns)

,race_id,単勝,複勝1,複勝2,複勝3,枠連,馬連,ワイド1,ワイド2,ワイド3,馬単,3連複,3連単
0,202009040202,420,160,720,200,1110,8340,2370,480,2510,10720,11500,76230


In [77]:
racer = race()
pd.DataFrame([racer.raptime(race_id)], columns=racer.raptimes_list)

,race_id,100m,200m,300m,400m,500m,600m,700m,800m,900m,1000m,1100m,1200m,1300m,1400m,1500m,1600m,1700m,1800m,1900m,2000m,2100m,2200m,2300m,2400m,2500m,2600m
0,202009040202,None,12.7,None,23.3,None,35.1,None,47.0,None,59.2,None,1:11.3,None,1:23.3,None,1:35.9,None,None,None,None,None,None,None,None,None,None


In [78]:
from operator import itemgetter
import pandas as pd

rows = driver2.find_element_by_id('Ninki').text.replace('\n', ' ').split(' ')
list(zip(*[iter(rows[9:])]*8))
b = [0, 1, 2, 3, 4, 5, 7]
columns = ['人気', '枠', '馬番', '馬名', '単勝オッズ', '複勝オッズ1', '複勝オッズ2']
data = [itemgetter(*b)(row) for row in list(zip(*[iter(rows[9:])]*8))]
pd.DataFrame(data, columns=columns)

,人気,枠,馬番,馬名,単勝オッズ,複勝オッズ1,複勝オッズ2
0,1,8,16,トクシーズファー,3.2,1.3,1.7
1,2,7,14,ズールー,3.9,1.7,2.4
2,3,1,2,スピードオブラブ,5.7,1.8,2.6
3,4,4,7,アーデントラブ,6.0,1.5,2.1
4,5,3,5,ユウチャファスト,8.3,1.7,2.5
5,6,2,3,バースデーギフト,18.0,3.8,5.9
6,7,2,4,ヘライア,24.3,6.0,9.3
7,8,6,12,ニットウムギ,24.4,4.1,6.2
8,9,4,8,アッタヴァンティ,37.3,7.5,11.7
9,10,6,11,デルマアイオライト,52.1,11.7,18.4


## ここよりしたは無視して

In [61]:
race_id = '202003070610'
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
html = requests.get('https://www.keibalab.jp/db/race/' + race_id + '/odds.html', headers=headers)
soup = BeautifulSoup(html.content, 'lxml')
odds_table = soup.find("table", {"id": 'oddsTanTable'})
# tmp = odds_table.find("tbody").findAll(["td"])
# elements = list(map(lambda x: x.text, tmp))
# list(zip(*[iter(elements)]*8))
columns = ['name', 'tan', 'huku1', 'huku2']
rows = []
for i, element in enumerate(odds_table.find("tbody").findAll(["td"])):
  if i%8 not in [0, 1, 5, 7]:
    rows.append(element.text)
list(zip(*[iter(rows)]*4))

[('テトラルキア', '15.4', '3.0', '5.0'),
 ('バンブトンハート', '98.0', '11.8', '20.9'),
 ('ロードアクシス', '34.8', '5.4', '9.3'),
 ('リョウランヒーロー', '42.5', '4.0', '6.8'),
 ('フィールザファラオ', '61.0', '7.1', '12.4'),
 ('エオリア', '12.1', '1.9', '3.0'),
 ('カナシバリ', '13.5', '2.1', '3.3'),
 ('ボードウォーク', '101.6', '16.5', '29.3'),
 ('オメガドラクロワ', '90.3', '10.7', '19.0'),
 ('エクリリストワール', '3.0', '1.2', '1.6'),
 ('アシャカトブ', '4.3', '1.4', '2.1'),
 ('レッドフレイ', '2.6', '1.2', '1.7')]

In [59]:
import pandas as pd

pd.DataFrame(list(zip(*[iter(rows)]*4)), columns=columns)

,name,tan,huku1,huku2
0,テトラルキア,15.4,3.0,5.0
1,バンブトンハート,98.0,11.8,20.9
2,ロードアクシス,34.8,5.4,9.3
3,リョウランヒーロー,42.5,4.0,6.8
4,フィールザファラオ,61.0,7.1,12.4
5,エオリア,12.1,1.9,3.0
6,カナシバリ,13.5,2.1,3.3
7,ボードウォーク,101.6,16.5,29.3
8,オメガドラクロワ,90.3,10.7,19.0
9,エクリリストワール,3.0,1.2,1.6
